In [15]:
import pyecharts.options as opts
from pyecharts.charts import Line
import pandas as pd
def visual_change(file):
    """
    对每日/周/月/季度/年涨跌幅前10名按涨/跌幅可视化输出
    stocks: 列表, 内部装股票名, 按涨/跌幅排序, 输入的是csv 文件, 横坐标是时间, 纵坐标为股票在此期限的涨跌幅
    period: 时间期限
    """
    line = Line()
    line.add_xaxis(xaxis_data=file.columns.to_list()[1:])
    for index, row in file.iterrows():
        line.add_yaxis(
            series_name = row["stock"],
            stack="总量",
            y_axis = row.to_list()[1:],
            label_opts=opts.LabelOpts(is_show=False),
        )
    line.set_global_opts(
        title_opts=opts.TitleOpts(title="前十股票涨跌幅"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        )
    line.render()
    return line

In [59]:
from pyecharts import options as opts
from pyecharts.charts import Kline
def candle_graph(stockfile,, stockname, start_date, end_date):
    """
    绘制蜡烛图
    stock: 输入csv 文件, [Date,Open,High,Low,Close,Volume]
    """

    date_list = stockfile["Date"].to_list()
    start_index = date_list.index(start_date)
    end_index = date_list.index(end_date)
    date_list = date_list[start_index:end_index+1]
    data = []
    for index, row in stockfile.iterrows():
        row = row.to_dict()
        if start_index <= index and index <= end_index:
            day_data = [row["Open"], row["Close"], row["High"], row["Low"]]
            data.append(day_data)
    c = (
    Kline()
    .add_xaxis(date_list)
    .add_yaxis("kline", data)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title=stockname),
    )
    .render(stockname+"kline_markline.html")
    )
    return

In [60]:
def visualisation_kline(stocks_name,period):
    """
    绘制每日/周/月/季度/年涨幅、跌幅前3/5/10/10/20名的月度/季度/年度/5年度蜡烛图（开盘价、收盘价等)
    stocks_name: 跌幅前3/5/10/10/20名的名单, 以列表的形式
    """
    
    for stockname in stocks_name:
        path = element+".csv"
        file2 = pd.read_csv(path)
        # calculate period function 尚未实现
        candle_graph(file2,stockname, "2016-03-14", "2016-04-14")

In [1]:
    import pandas as pd
    import pymysql
    from get_pricedata import *
    from creat_database import *
    import datetime
    from datetime import datetime
    import numpy as np
    from change_table import *
    db = conn_sql("139.9.222.28",'lulu','lulu666')
    cursor = db.cursor()
    def sort_table(period, top_rank, rise = True):
        """
        period: 排序的周期
        top_rank: 取出前多少名， 以及涨跌幅
        rise: 涨/跌幅， 默认涨
        """
        df = read_change_table(db, cursor, period)
        df.set_index("id", inplace= True)
        df1 = df.groupby("update_day")
        all_groups = df1.groups.keys()
        col_name = "{}_change_percent".format(period)
        for group in all_groups:
            df2 = df1.get_group(group).replace("nan", np.nan) 
            df2.dropna(subset = [column], inplace =True)
            if not df2.empty:
                df2[col_name] = df2[col_name].astype(float)
                if rise:    
                    # 涨幅前几（根据输入的top rank)
                    sort_df2 = df2.sort_values(by = [col_name], ascending = False).head(top_rank)
                    pd.DataFrame.to_csv("涨幅_{}_{}.csv".format(period, top_rank))
                else:
                    # 跌幅前几（根据输入的top rank)
                    reverse_sort_df2 = df2.sort_values(by =[col_name], ascending =True).head(top_rank)
                    pd.DataFrame.to_csv("跌幅_{}_{}.csv".format(period, top_rank))

change_db数据库读取成功
        id company_name update_day  year_change_percent
0        1           JG       2018                  nan
1        2           JG       2019  -21.387277565102885
2        3           JG       2020  -60.882352776180916
3        4           JG       2021    22.18044718540154
4        5         FEDU       2017                  nan
...    ...          ...        ...                  ...
1460  1461         MOMO       2019    -8.01127117107059
1461  1462         MOMO       2020   49.277899522545155
1462  1463         MOMO       2021   -59.99706768218954
1463  1464         EBON       2020                  nan
1464  1465         EBON       2021    74.54068753911848

[1465 rows x 4 columns]
year_change_table表读取成功
